# Funkcionalno programiranje

Amer Hasanović

## Computation Expressions (CE)

Određeni proračuni, zbog svojih karakteristika, neelegatno i/ili teško se mogu formulisati korištenjem standardne sintakse programskog jezika.

Funkcionalni programski jezici često omogućavaju programerima da prošire sintaksu jezika i simplificiraju specifikaciju takvih problema u alternativnim formama. 

* `F#` tu mogućnost pruža kroz `CE`,
* `Haskell` kroz sličan koncept koji se zove `Monad`.


In [2]:
open System

let tryParse (s:string) = 
  let mutable res = 0
  if Int32.TryParse(s, &res) then
      Some res
  else
      None

let num () = System.Console.ReadLine() |> tryParse



In [3]:
let sum ao bo co = 
  match ao with 
  | Some a ->
     match bo with 
     | Some b -> 
       match co with
       | Some c -> Some (a + b + c)
       | None -> None
     | None -> None
  | None -> None

//sum (num ()) (num ()) (num ()) |> printfn "%A"

In [4]:
type MaybeBuilder() =

  member this.Bind(o, f) = 
    match o with
    | Some x -> f x 
    | None -> None

  member this.Return(x) = Some x

let maybe = MaybeBuilder()

In [5]:
let sum0 ao bo co = 
  match ao with 
  | Some a ->
     match bo with 
     | Some b -> 
       match co with
       | Some c -> maybe.Return(a + b + c)
       | None -> None
     | None -> None
  | None -> None

In [6]:
let sum1 ao bo co = 
  match ao with 
  | Some a ->
     match bo with 
     | Some b -> 
         maybe.Bind(co, fun c -> 
           maybe.Return(a+b+c)
         )
     | None -> None
  | None -> None

In [7]:
let sum2 ao bo co = 
  match ao with 
  | Some a -> 
      maybe.Bind( bo, fun b -> 
        maybe.Bind(co, fun c -> 
          maybe.Return(a+b+c)
        )
      )
  | None -> None

In [8]:
let sum3 ao bo co =
  maybe.Bind(ao, fun a ->
    maybe.Bind(bo, fun b ->
      maybe.Bind(co, fun c ->
        maybe.Return (a + b + c)
      )
    )
  )

In [9]:
let sum4 ao bo co =
  maybe.Bind(ao, fun a ->
  maybe.Bind(bo, fun b ->
  maybe.Bind(co, fun c ->
  maybe.Return (a + b + c)
  )))

Neka je:

```txt
maybe.Return x <==> maybe { return x }
```

In [10]:
let sum5 ao bo co = 
  maybe.Bind(ao, fun a ->
  maybe.Bind(bo, fun b ->
  maybe.Bind(co, fun c ->
    maybe { return a + b + c }
  )))

Neka je:

```txt
maybe.Bind(xo, fun x ->           maybe {
  yo                        <==>     let! x = xo
)                                    yo
                                  }
```

In [11]:
let sum6 ao bo co = 
  maybe.Bind(ao, fun a ->
  maybe.Bind(bo, fun b ->
  maybe {
      let! c = co
      return a + b + c 
  }
  ))

In [ ]:
let sum7 ao bo co = 
  maybe.Bind(ao, fun a ->
  maybe {
      let! b = bo
      let! c = co
      return a + b + c 
  }
  )

In [ ]:
let sum8 ao bo co = maybe {
    let! a = ao
    let! b = bo
    let! c = co
    return a + b + c
}

// sum8 (num ()) (num ()) (num ()) |> printfn "%A"



In [13]:
let sumL ls = 
  ls 
  |> List.map tryParse 
  |> List.fold (fun ao eo -> maybe { 
                  let! a = ao
                  let! e = eo
                  return a + e
                }) (Some 0)

// List.init 3 (fun _ -> System.Console.ReadLine()) 
// |> sumL
// |> printfn "%A" 

## Asinhroni program

Predstavlja program u kojem jedan ili više dijelova proračuna, iz nekih razloga, zahtijevaju odgodu.

Najčešći primjeri su programi sa intenzivnim `IO` operacijama, npr:

* GUI
* Web aplikacije frontend i bekend


Za potrebe asinhronog programiranja `F#` definira `async` CE.

`async` CE proizvodi vrijednost tipa `Async<'a>`, koja reprezentira neki proračun koji kad se izvrši proizvodi vrijednost tipa `'a`.

In [ ]:
let comp f x (t:int) = async {
    do! Async.Sleep t
    return (f x)
}

let c1s = comp (fun x -> printfn "%A" x) "1s" 1000
let c4s = comp (fun x -> printfn "%A" x) "4s" 4000

open System

let bench ce = async {
  let n = DateTime.Now
  let! _ = ce
  return (DateTime.Now-n).TotalMilliseconds
}

let cseq = Async.Sequential [c1s; c4s]
let cpar = Async.Parallel [c1s; c4s]

printfn "Start1"
bench cseq 
|> Async.RunSynchronously 
|> printfn "Seq. time: %A" 

printfn "Start2"
bench cpar 
|> Async.RunSynchronously 
|> printfn "Par. time: %A" 


let inf_comp ct = async {
  while true do
    do! ct
}

module Async =

  let map f op = async {
      let! x    = op
      return f x
  }

let inf_cpar = 
  [inf_comp c1s; inf_comp c4s ] 
  |> Async.Parallel
  |> Async.map (fun _ -> Some ()) 

let c5s = comp (fun _ -> Some ()) () 5000

printfn "Start3"
bench ( Async.Choice [inf_cpar; c5s] )
|> Async.RunSynchronously 
|> printfn "Total time: %A" 

In [ ]:
let comp_expr n s = async {
  if n % 2 <> 0 then 
    failwith s
  return n  
}

let exprs = [ 
  comp_expr 9 "1"
  comp_expr 3 "2"
]

try
  exprs 
  |> Async.Parallel 
  |> Async.RunSynchronously
  |> printfn "%A"
with 
 | Failure(s) -> s |> printfn "%s"

In [ ]:
open System

let ct1 = async {
  use! c = Async.OnCancel( 
    fun () -> printfn "ct1 Canceled"
  )

  while true do 
    do! Async.Sleep(2000)
    DateTime.Now |> printfn "%A"

}

let ct2 = async {
  use! c = Async.OnCancel(
    fun () -> printfn "ct2 Canceled"
  )

  let! test = Async.StartChild ct1

  while true do 
    do! Async.Sleep(1000)
    printfn "ct2"
}

let ts = new System.Threading.CancellationTokenSource()
printfn "Start"
let r = Async.Start(ct2, cancellationToken=ts.Token)    
printfn "Waiting"
Console.ReadKey() |> ignore
ts.Cancel()
printfn "Done!"

In [ ]:
open System
let primes = [ 2; 3; 5; 7; 4; 9 ]


let computations = [ 
  for i in primes do
    async {
      do! Async.Sleep(System.Random().Next(2000, 9000))
      return if i % 2 <> 0 then Some(i) else None
    }
]

let ct = new System.Threading.CancellationTokenSource()

let c1 = async {
    System.Console.ReadKey() |> ignore
    ct.Cancel()
}

printfn "Start"
Async.Start c1
try 
  Async.RunSynchronously( 
    computation = (computations |> Async.Choice), 
    cancellationToken = ct.Token 
  )
  |> function
    | Some (i) -> printfn $"{i}"
    | None -> printfn "No Result"
with
  | :? System.OperationCanceledException -> printfn "Canceled"